In [1]:
import spacy
import re
import numpy as np
import pandas as pd
import pickle
import gensim
from gensim.models.word2vec import Word2Vec
import nltk


#sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import log_loss, recall_score, precision_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier


import xgboost as xgb

nlp = spacy.load('en')

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


combined pcfg

In [18]:
df_jo = pd.read_csv("data/features/data_joel.csv", sep = ",")
df_jy = pd.read_csv("data/features/data_jingyang.csv", sep = ",")
df_wj = pd.read_csv("data/features/data_weijie.csv", sep = ",")
df_gh1 = pd.read_csv("data/features/data_3500to4000.csv", sep = ",")
df_gh2 = pd.read_csv("data/features/data_4000to5000.csv", sep = ",")
df_gh3 = pd.read_csv("data/features/data_5000to6000.csv", sep = ",")
df_gh4 = pd.read_csv("data/features/data_6000to7000.csv", sep = ",")

In [9]:
df_ori = pd.read_csv("dataset.csv", sep = ",")

In [19]:
new_df = pd.concat([df_jy, df_gh1, df_gh2, df_gh3, df_gh4, df_jo, df_wj])

In [39]:
new_df.iloc[:,3:].to_csv("pcfg_output.csv", index = False)

combined postag

In [3]:
df_pt1 = pd.read_csv("postag_output_0_5000.csv", sep = ",")
df_pt2 = pd.read_csv("postag_output_5000_10000.csv", sep = ",")
df_pt3 = pd.read_csv("postag_output_10000_end.csv", sep = ",")

In [4]:
new_df_pt = pd.concat([df_pt1, df_pt2, df_pt3])

In [6]:
new_df_pt.shape

(15679, 12)

In [7]:
new_df_pt.head()

,total_word_count,avg_word_length,lexical_diversity,repetition_top,repetition_all,NNP_percent,NNPS_percent,noun_percent,verb_percent,part_percent,det_percent,unknown_or_foreign_percent
0,1549,4.571336,0.371853,0.339574,0.130808,0.111924,0.007312,0.137795,0.172103,0.028121,0.113611,0.0
1,488,4.534836,0.561475,0.315574,0.108682,0.099822,0.003565,0.146168,0.171123,0.030303,0.117647,0.0
2,443,4.458239,0.577878,0.347630,0.107576,0.099174,0.004132,0.130165,0.171488,0.008264,0.128099,0.0
3,61,4.344262,0.836066,0.491803,0.136849,0.169231,0.015385,0.138462,0.184615,0.030769,0.138462,0.0
4,505,4.879208,0.534653,0.328713,0.109228,0.053819,0.003472,0.262153,0.171875,0.008681,0.064236,0.0


In [8]:
new_df_pt.to_csv("postag_output.csv", index = False)

combine all

In [11]:
df_ori = pd.read_csv("dataset.csv", sep = ",")

In [13]:
x = df_ori.text
y = df_ori.label

In [24]:
df_pcfg = pd.read_csv("data/features/pcfg_output.csv", sep = ",")
df_postag = pd.read_csv("data/features/postag_output.csv", sep = ",")
df_tfidf = pd.read_csv("data/features/tfidf_output.csv", sep = ",")
df_w2v = pd.read_csv("data/features/w2v_output.csv", sep = ",")

In [25]:
df_pcfg.head(1)

,total_sentences,num_sentences_with_adj_in_phrase,num_sentences_with_adv_in_phrase
0,76,33,19


In [26]:
df_postag.head(1)

,total_word_count,avg_word_length,lexical_diversity,repetition_top,repetition_all,NNP_percent,NNPS_percent,noun_percent,verb_percent,part_percent,det_percent,unknown_or_foreign_percent
0,1549,4.571336,0.371853,0.339574,0.130808,0.111924,0.007312,0.137795,0.172103,0.028121,0.113611,0.0


In [27]:
df_tfidf.head(1)

,prob_nb,prob_logreg
0,0.003172,0.2068


In [28]:
df_w2v.head(1)

,prob_nb,prob_logreg,prob_svc,prob_rforest
0,0.000657,0.441064,0.348825,0.5


In [38]:
final_comb_df = pd.concat([df_pcfg, df_postag, df_tfidf, df_w2v,y], axis = 1)

In [46]:
final_comb_df.columns = ['total_sentences', 'num_sentences_with_adj_in_phrase',
       'num_sentences_with_adv_in_phrase', 'total_word_count',
       'avg_word_length', 'lexical_diversity', 'repetition_top',
       'repetition_all', 'NNP_percent', 'NNPS_percent', 'noun_percent',
       'verb_percent', 'part_percent', 'det_percent',
       'unknown_or_foreign_percent', 'tfidf_prob_nb', 'tfidf_prob_logreg', 'w2v_prob_nb',
       'w2v_prob_logreg', 'w2v_prob_svc', 'w2v_prob_rforest', 'is_fake']

In [48]:
final_comb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15679 entries, 0 to 15678
Data columns (total 22 columns):
total_sentences                     15679 non-null int64
num_sentences_with_adj_in_phrase    15679 non-null int64
num_sentences_with_adv_in_phrase    15679 non-null int64
total_word_count                    15679 non-null int64
avg_word_length                     15679 non-null float64
lexical_diversity                   15679 non-null float64
repetition_top                      15679 non-null float64
repetition_all                      15679 non-null float64
NNP_percent                         15679 non-null float64
NNPS_percent                        15679 non-null float64
noun_percent                        15679 non-null float64
verb_percent                        15679 non-null float64
part_percent                        15679 non-null float64
det_percent                         15679 non-null float64
unknown_or_foreign_percent          15679 non-null float64
tfidf_prob_nb 

In [51]:
final_comb_df.head()

,total_sentences,num_sentences_with_adj_in_phrase,num_sentences_with_adv_in_phrase,total_word_count,avg_word_length,lexical_diversity,repetition_top,repetition_all,NNP_percent,NNPS_percent,...,part_percent,det_percent,unknown_or_foreign_percent,tfidf_prob_nb,tfidf_prob_logreg,w2v_prob_nb,w2v_prob_logreg,w2v_prob_svc,w2v_prob_rforest,is_fake
0,76,33,19,1549,4.571336,0.371853,0.339574,0.130808,0.111924,0.007312,...,0.028121,0.113611,0.0,3.171641e-03,0.206800,6.571201e-04,0.441064,0.348825,0.5,0
1,27,12,8,488,4.534836,0.561475,0.315574,0.108682,0.099822,0.003565,...,0.030303,0.117647,0.0,9.980483e-01,0.637621,9.999960e-01,0.822067,0.877119,0.6,1
2,20,13,3,443,4.458239,0.577878,0.347630,0.107576,0.099174,0.004132,...,0.008264,0.128099,0.0,9.972521e-01,0.865423,1.000000e+00,0.977274,0.996848,0.9,1
3,3,1,0,61,4.344262,0.836066,0.491803,0.136849,0.169231,0.015385,...,0.030769,0.138462,0.0,6.997447e-10,0.042518,1.646796e-09,0.021398,0.001886,0.0,0
4,50,20,3,505,4.879208,0.534653,0.328713,0.109228,0.053819,0.003472,...,0.008681,0.064236,0.0,1.000000e+00,0.937735,9.999951e-01,0.826256,0.876790,1.0,1


In [52]:
final_comb_df.to_csv("features_and_label.csv",index = False)